In [1]:
import gym
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


#### Implementação original:
https://ieeexplore-ieee-org.ezp2.lib.umn.edu/stamp/stamp.jsp?tp=&arnumber=6313077

#### Baseado no jupyter-notebook: 
https://github.com/gsurma/cartpole/blob/master/cartpole.py

In [2]:

ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995


In [ ]:

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Tomar Decisões Aleatorias em certos momentos
        if np.random.rand() < self.exploration_rate:
            print("Inovou.")
            return random.randrange(self.action_space)
        # Prever q_value dado o estado atual
        q_values = self.model.predict(state)
        # Retornar acao
        return np.argmax(q_values[0])

    def experience_replay(self):
        # Retreinar modelo com novas informacoes
        if len(self.memory) < BATCH_SIZE:
            return
        # Selecionar aleatoriamente momentos na memoria
        batch = random.sample(self.memory, BATCH_SIZE)
        # Para cada momento na memoria
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                # Caso o estado não tenha sido terminal
                # Atualiza q_values. Ver imagem: https://wikimedia.org/api/rest_v1/media/math/render/svg/678cb558a9d59c33ef4810c9618baf34a9577686
                # Novo q_value é Recompensa + Com a ação prevista para o próximo estado * Gamma
                # Se a recompensa for negativa, o q_value para esta ação no proximo estado é diminuido.
                # Se a recompensa for positiva, o q_value para esta ação no proximo estado é aumentado
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0) #Online learning - O modelo é retreinado a cada momento lembrado
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

###  Ideia da solução baseada em reforço

<img src="https://miro.medium.com/max/1400/1*mPGk9WTNNvp3i4-9JFgD3w.png" width=600 />

In [4]:
def cartpole():
    #Iniciar o ambiente de treino
    env = gym.make("CartPole-v1")
    #Salvar tamanho do espaço observado - Neste caso é 4 (Posicao do carro, Velocidade do Carro, Angulo com vertical, Velocidade do Topo)
    observation_space = env.observation_space.shape[0]
    # Salvar tamanho do espaco de acao. - Neste caso é 2 (esquerda ou direita)
    action_space = env.action_space.n
    # Inicializa nosso resolvedor
    dqn_solver = DQNSolver(observation_space, action_space)
    episode = 0
    while True:
        steps = 0
        episode += 1
        # Reinicia o ambiente a cada interacao
        state = env.reset()
        # Transforma o vetor state em matrix
        state = np.reshape(state, [1, observation_space])
        
        while True:
            # Desenha o ambiente
            env.render()
            # Preve melhor proxima acao dado estado corrente
            action = dqn_solver.act(state)
            # Realiza acao e atualiza status
            state_next, reward, terminal, info = env.step(action)
            # Se o ambiente não foi finalizado a recompensa é positiva, se não negativa.
            reward = reward if not terminal else -reward
            # Transforma o vetor state em matrix
            state_next = np.reshape(state_next, [1, observation_space])
            q_values = dqn_solver.model.predict(state)
            print("\tSTEP: {} - Valores Q: {}\n\tAcao: {} Recompensa: {} Finalizado:{}\n\tPROXIMO ESTADO- Pos: {:.2f} Car Vel:{:.2f} Angulo:{:.2f} Vel Top:{:.2f}".format(steps,q_values, action, reward, terminal, state_next[0][0],state_next[0][1],state_next[0][2],state_next[0][3]))
            # Faz com que a acao seja lembrada.
            dqn_solver.remember(state, action, reward, state_next, terminal)
            # Re-treina nosso modelo com a nova experiencia
            dqn_solver.experience_replay()
            state = state_next
            steps += 1
            if terminal:
                break
        print("Episode: {} - Passos no ultimo episodio: {}".format(episode, steps))


In [ ]:
cartpole()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Inovou.

	STEP: 0 - Valores Q: [[ 0.00918258 -0.01768184]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.03 Car Vel:0.24 Angulo:-0.03 Vel Top:-0.28
Inovou.
	STEP: 1 - Valores Q: [[ 0.10420085 -0.0673541 ]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.03 Car Vel:0.43 Angulo:-0.04 Vel Top:-0.58
Inovou.
	STEP: 2 - Valores Q: [[ 0.2080712  -0.14495316]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.04 Car Vel:0.24 Angulo:-0.05 Vel Top:-0.30
Inovou.
	STEP: 3 - Valores Q: [[ 0.10998864 -0.07334565]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:0.05 Angulo:-0.05 Vel Top:-0.02
Inovou.
	STEP: 4 - Valores Q: [[ 0.01074629 -0.03097236]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:-0.15 Angulo:-0.05 Vel Top:0.25
Inovou.
	STEP: 5 - Valores Q: [[-0.00774664 -0.02859567]]
	Acao: 1 Recompensa: 1.0 

Inovou.
	STEP: 14 - Valores Q: [[ 9.379953 10.053787]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.01 Car Vel:-0.22 Angulo:-0.03 Vel Top:0.32
Inovou.
	STEP: 15 - Valores Q: [[ 9.581025 10.305482]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.01 Car Vel:-0.42 Angulo:-0.03 Vel Top:0.61
	STEP: 16 - Valores Q: [[10.75552  11.011463]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.02 Car Vel:-0.22 Angulo:-0.01 Vel Top:0.31
Inovou.
	STEP: 17 - Valores Q: [[10.621357 10.787207]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.03 Car Vel:-0.03 Angulo:-0.01 Vel Top:0.01
Inovou.
	STEP: 18 - Valores Q: [[11.059722 11.168209]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.03 Car Vel:0.17 Angulo:-0.01 Vel Top:-0.29
Inovou.
	STEP: 19 - Valores Q: [[11.548779 11.112829]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.02 Car Vel:-0.03 Angulo:-0.01 Vel Top:0.00
Inovou.
	STEP: 20 

Inovou.
	STEP: 27 - Valores Q: [[252.35066 233.6985 ]]
	Acao: 0 Recompensa: -1.0 Finalizado:True
	PROXIMO ESTADO- Pos: -0.20 Car Vel:-1.56 Angulo:0.22 Vel Top:2.25
Episode: 3 - Passos no ultimo episodio: 28
Inovou.
	STEP: 0 - Valores Q: [[177.92241 157.79573]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.01 Car Vel:-0.21 Angulo:0.00 Vel Top:0.25
Inovou.
	STEP: 1 - Valores Q: [[184.32889 165.70001]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.01 Car Vel:-0.41 Angulo:0.01 Vel Top:0.54
Inovou.
	STEP: 2 - Valores Q: [[188.48584 172.69763]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.02 Car Vel:-0.60 Angulo:0.02 Vel Top:0.83
	STEP: 3 - Valores Q: [[196.12509 181.56467]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.03 Car Vel:-0.80 Angulo:0.03 Vel Top:1.13
	STEP: 4 - Valores Q: [[205.74913 191.55606]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.05 Car Vel:-0.99 Angulo:0.06 Vel Top

	STEP: 11 - Valores Q: [[109.725586 107.884674]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.11 Car Vel:-1.15 Angulo:0.17 Vel Top:2.03
Inovou.
	STEP: 12 - Valores Q: [[111.7962   110.908424]]
	Acao: 1 Recompensa: -1.0 Finalizado:True
	PROXIMO ESTADO- Pos: -0.13 Car Vel:-0.96 Angulo:0.21 Vel Top:1.79
Episode: 7 - Passos no ultimo episodio: 13
Inovou.
	STEP: 0 - Valores Q: [[107.02667 102.06776]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:0.23 Angulo:-0.03 Vel Top:-0.32
	STEP: 1 - Valores Q: [[105.051834  99.382645]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:0.04 Angulo:-0.04 Vel Top:-0.03
	STEP: 2 - Valores Q: [[104.633095  99.805954]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:-0.16 Angulo:-0.04 Vel Top:0.25
Inovou.
	STEP: 3 - Valores Q: [[103.025665  98.96422 ]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:-0.35 Angulo:-0.04 Vel 

	PROXIMO ESTADO- Pos: 0.02 Car Vel:1.21 Angulo:-0.11 Vel Top:-1.82
	STEP: 6 - Valores Q: [[61.951485 62.777493]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:1.41 Angulo:-0.14 Vel Top:-2.14
Inovou.
	STEP: 7 - Valores Q: [[64.14903 65.01507]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.07 Car Vel:1.60 Angulo:-0.19 Vel Top:-2.48
	STEP: 8 - Valores Q: [[67.93288 68.83795]]
	Acao: 1 Recompensa: -1.0 Finalizado:True
	PROXIMO ESTADO- Pos: 0.11 Car Vel:1.80 Angulo:-0.24 Vel Top:-2.82
Episode: 10 - Passos no ultimo episodio: 9
	STEP: 0 - Valores Q: [[53.821136 54.881317]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.01 Car Vel:0.17 Angulo:-0.00 Vel Top:-0.25
Inovou.
	STEP: 1 - Valores Q: [[53.464405 53.606956]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: -0.00 Car Vel:0.37 Angulo:-0.01 Vel Top:-0.55
Inovou.
	STEP: 2 - Valores Q: [[53.175186 52.820576]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROX

...

	STEP: 53 - Valores Q: [[23.802792 23.966095]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.11 Car Vel:-0.04 Angulo:0.06 Vel Top:0.32
	STEP: 54 - Valores Q: [[23.69576  23.974197]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.11 Car Vel:0.15 Angulo:0.07 Vel Top:0.04
	STEP: 55 - Valores Q: [[23.62678  23.836536]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.11 Car Vel:0.35 Angulo:0.07 Vel Top:-0.23
	STEP: 56 - Valores Q: [[23.822035 23.73298 ]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.12 Car Vel:0.15 Angulo:0.07 Vel Top:0.09
	STEP: 57 - Valores Q: [[23.713402 23.87494 ]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.12 Car Vel:0.35 Angulo:0.07 Vel Top:-0.18
	STEP: 58 - Valores Q: [[23.840458 23.907442]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.13 Car Vel:0.54 Angulo:0.06 Vel Top:-0.45
	STEP: 59 - Valores Q: [[23.518587 23.376703]]
	Acao: 0 Recompensa: 1.0 

	PROXIMO ESTADO- Pos: 0.37 Car Vel:-0.05 Angulo:0.09 Vel Top:0.45
	STEP: 106 - Valores Q: [[24.480148 24.617254]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.37 Car Vel:0.15 Angulo:0.10 Vel Top:0.18
	STEP: 107 - Valores Q: [[24.252707 24.117718]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.37 Car Vel:-0.05 Angulo:0.10 Vel Top:0.50
	STEP: 108 - Valores Q: [[22.768446 22.434074]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.37 Car Vel:-0.24 Angulo:0.11 Vel Top:0.83
	STEP: 109 - Valores Q: [[20.842875 21.28495 ]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.36 Car Vel:-0.05 Angulo:0.13 Vel Top:0.57
	STEP: 110 - Valores Q: [[21.71499  22.292702]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.36 Car Vel:0.14 Angulo:0.14 Vel Top:0.32
	STEP: 111 - Valores Q: [[22.259058 22.878027]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.36 Car Vel:0.34 Angulo:0.14 Vel Top:0.07
	

	PROXIMO ESTADO- Pos: 1.09 Car Vel:0.89 Angulo:-0.00 Vel Top:-0.17
	STEP: 159 - Valores Q: [[20.436886 20.240507]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.11 Car Vel:0.70 Angulo:-0.00 Vel Top:0.13
	STEP: 160 - Valores Q: [[19.61482  19.669685]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.13 Car Vel:0.89 Angulo:-0.00 Vel Top:-0.17
	STEP: 161 - Valores Q: [[19.482048 19.194672]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.14 Car Vel:0.70 Angulo:-0.01 Vel Top:0.12
	STEP: 162 - Valores Q: [[19.07711 19.56171]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.16 Car Vel:0.89 Angulo:-0.00 Vel Top:-0.17
	STEP: 163 - Valores Q: [[19.4459   19.549435]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.17 Car Vel:1.09 Angulo:-0.01 Vel Top:-0.46
	STEP: 164 - Valores Q: [[19.10692  18.353134]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.20 Car Vel:0.89 Angulo:-0.02 Vel Top:-

	STEP: 214 - Valores Q: [[20.122913 20.233961]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.94 Car Vel:1.26 Angulo:0.08 Vel Top:-0.26
	STEP: 215 - Valores Q: [[20.1492  19.58871]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.97 Car Vel:1.06 Angulo:0.07 Vel Top:0.06
	STEP: 216 - Valores Q: [[19.319843 19.497372]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 1.99 Car Vel:1.26 Angulo:0.07 Vel Top:-0.21
	STEP: 217 - Valores Q: [[19.580359 19.003351]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 2.01 Car Vel:1.06 Angulo:0.07 Vel Top:0.10
	STEP: 218 - Valores Q: [[14.850896 13.939611]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 2.03 Car Vel:0.87 Angulo:0.07 Vel Top:0.41
	STEP: 219 - Valores Q: [[9.495067 8.99868 ]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 2.05 Car Vel:0.67 Angulo:0.08 Vel Top:0.73
	STEP: 220 - Valores Q: [[7.802731 8.272281]]
	Acao: 1 Recompensa: 1.0 F

	STEP: 31 - Valores Q: [[23.95393  23.720371]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.04 Car Vel:0.04 Angulo:0.08 Vel Top:0.17
	STEP: 32 - Valores Q: [[23.976074 24.22426 ]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.04 Car Vel:0.23 Angulo:0.08 Vel Top:-0.09
	STEP: 33 - Valores Q: [[23.057169 23.42952 ]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:0.43 Angulo:0.08 Vel Top:-0.36
	STEP: 34 - Valores Q: [[22.928371 23.127352]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.05 Car Vel:0.62 Angulo:0.08 Vel Top:-0.63
	STEP: 35 - Valores Q: [[23.404238 23.484251]]
	Acao: 1 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.07 Car Vel:0.81 Angulo:0.06 Vel Top:-0.89
	STEP: 36 - Valores Q: [[24.107265 24.082092]]
	Acao: 0 Recompensa: 1.0 Finalizado:False
	PROXIMO ESTADO- Pos: 0.08 Car Vel:0.62 Angulo:0.05 Vel Top:-0.58
	STEP: 37 - Valores Q: [[23.972454 24.19881 ]]
	Acao: 1 Recompensa: 1.0